In [2]:
import pygame
import random
import math
import time
import queue

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
conveyor_belts = [
    [(1200, 600), (100, 600)],
    [(100, 550), (800, 550)],
    [(200, 500), (800, 500)],
    [(200, 450), (700, 450)],
    [(300, 400), (700, 400)],
    [(300, 350), (600, 350)],
    [(100, 250), (900, 250)],
    [(100, 600), (100, 200)],
    [(200, 500), (200, 200)],
    [(300, 400), (300, 200)],
    [(600, 350), (600, 200)],
    [(700, 450), (700, 200)],
    [(800, 550), (800, 200)],
    [(900, 600), (900, 250)],
    [(580, 200), (600, 250)],
    [(620, 200), (600, 250)],
    [(320, 200), (300, 250)],
    [(280, 200), (300, 250)],
    [(680, 200), (700, 250)],
    [(720, 200), (700, 250)],
    [(180, 200), (200, 250)],
    [(220, 200), (200, 250)],
    [(780, 200), (800, 250)],
    [(820, 200), (800, 250)],
    [(80, 200), (100, 250)],
    [(120, 200), (100, 250)],
]

points = [
    (100, 250), (100, 550), (100, 600),
    (200, 250), (200, 450), (200, 500),
    (300, 250), (300, 350), (300, 400),
    (600, 250), (600, 350), (700, 250),
    (700, 400), (700, 450), (800, 250),
    (800, 500), (800, 550), (900, 250),
    (900, 600)
]

gates_entry_positions = [
    (600, 250),  # Gate 2
    (300, 250),  # Gate 3
    (700, 250),  # Gate 4
    (200, 250),  # Gate 5
    (800, 250),  # Gate 6
    (100, 250),  # Gate 7
]

pickup_gates_positions = [
    (600, 200),  # Gates 2
    (580, 200),
    (620, 200),
    (300, 200),  # Gates 3
    (320, 200),
    (280, 200),
    (700, 200),  # Gates 4
    (680, 200),
    (720, 200),
    (200, 200),  # Gates 5
    (180, 200),
    (220, 200),
    (800, 200),  # Gates 6
    (780, 200), 
    (820, 200),
    (100, 200),  # Gates 7
    (80, 200),
    (120, 200),
]


In [6]:
class Path:
    
    def __init__(self, name, points):
        self.name = name  
        self.points = points
    
paths = {
    cluster: Path(f"{cluster}", points) for cluster, points in {
        7: [(900, 600), (100, 600), (100, 550), (100, 250)],
        6: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (800, 250)], 
        5: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (200, 250)], 
        4: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (700, 450), 
            (700, 400), (700, 250)],
        3: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (700, 450), 
            (700, 400), (300, 400), (300, 350), (300, 250)],
        2: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (700, 450), 
            (700, 400), (300, 400), (300, 350), (600, 350), (600, 250)]
    }.items()
}

paths_2 = {
  cluster: Path(f"{cluster}", points) for cluster, points in {
        7: [(900, 250), (900, 600)],
        6: [(900, 250), (900, 600)], 
        5: [(900, 250), (900, 600)], 
        4: [(900, 250), (900, 600)],
        3: [(900, 250), (900, 600)],
        2: [(900, 250), (900, 600)]
    }.items()
}

def generate_random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

cluster_sizes = [2, 3, 4, 5, 6, 7] 

cluster_colors = {size: generate_random_color() for size in cluster_sizes}

#bag_image = pygame.image.load("ALAA2.png")  # Load the image
#bag_image = pygame.transform.scale(bag_image, (30, 30))  # Resize if needed



In [8]:
class Bag:  
        
    def __init__(self, bag_id, reservation_id, cluster_size):
        self.ready = False
        self.path = []
        self.bag_id = bag_id
        self.reservation = reservation_id
        self.cluster_size = cluster_size
        self.position = (1200, 600)
        self.assigned_to_gate = False
        self.gate = None
        self.speed = 2
        self.target = None 
        self.color = cluster_colors[cluster_size]


    def set_ready(self):
        self.ready = True   

    def set_path(self, path):
        self.path = path.points[:]  
        if self.path:
            self.target = self.path.pop(0) 

    def set_gate(self, pickup_gate):
        self.gate = pickup_gate

    def append_path(self, path):
        if isinstance(path, Path):  
            self.path.extend(path.points)  
        elif isinstance(path, list):  
            self.path.extend(path)  
        elif isinstance(path, tuple):
            self.path.append(path)
        
        if not self.target and self.path:
            self.target = self.path.pop(0)  # Get it moving


    def has_reached_last_point(self):
        if not self.path:
            return True 
    
        last_x, last_y = self.path[-1]    
        return abs(self.position[0] - last_x) < 2 and abs(self.position[1] - last_y) < 10

    
    def move(self):
        if self.target:
            x, y = self.position
            tx, ty = self.target
    
            dx = tx - x
            dy = ty - y
            distance = (dx**2 + dy**2) ** 0.5
    
            if distance <= self.speed:
                self.position = self.target  
                if self.path:
                    self.target = self.path.pop(0)  
                else:
                    self.target = None  
            else:
                # Move towards the target incrementally
                self.position = (x + (dx / distance) * self.speed, 
                                 y + (dy / distance) * self.speed)


In [10]:
class Gate:
    
    def __init__(self, name, gates, position):
        self.name = name 
        self.gates = gates
        self.position = position
        self.res_queue = queue.Queue() 

    def add_res(self, reservation):
        self.res_queue.put(reservation)

    def max_loading(self):
        for gate in self.gates:
            gate.set_max(self.name)

    def search(self, bag):
        for i in range(3):
            if(self.gates[i].reservation == bag.reservation):
                return i
        for i in range(3):
            if(self.gates[i].reservation == None):
                return i
        return -1
   

In [12]:
class PickupGate:
    
    def __init__(self, name, position):
        self.name = name 
        self.position = position
        self.reservation = None
        self.load = 0
        self.max_load = 0
        self.bags = []
        self.full = False 
        
    def set_reservation(self, reservation):
        if self.reservation is None:  
            self.reservation = reservation
            return True
        else:
            return False

    def add_luggage(self, bag):
        if bag.reservation == self.reservation:
            self.bags.append(bag)
            self.load += 1

    def set_max(self, max):
        self.max_load = max
            
    def is_full(self):
        if self.load == self.max_load:
            return True
        return False

    def clear_luggage(self):
        removed_bags = self.bags if self.bags else []  
        self.reservation = None
        self.load = 0
        self.bags = None 
        return removed_bags  


In [14]:
gate_entries = []
k = 0
for i in range(6):
    gates = []
    for j in range(3):
        gates.append(PickupGate(j+1, pickup_gates_positions[k]))
        k += 1
    gate_entries.append(Gate(i+2, gates, gates_entry_positions[i]))

for gate in gate_entries:
    gate.max_loading()



conveyor_belt = []

reservations = []

num_reservations = 30 
bag_id = 1


for reservation_id in range(1, num_reservations + 1):  
    cluster_size = random.randint(2, 7)  
    reservations.append((reservation_id, cluster_size))

    for _ in range(cluster_size):
        conveyor_belt.append(Bag(bag_id, reservation_id, cluster_size))
        bag_id += 1  

random.shuffle(conveyor_belt)

In [16]:
pygame.init()

width, height = 1600, 800
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("SLHS Visualization")

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

In [17]:
def gates_full(gates):
    fullnesses = []
    for gate in gates:
        for pickup_gate in gate.gates:
            if pickup_gate.is_full:
                fullnesses.append(True)
            else:
                fullnesses.append(False)
    return fullnesses

In [20]:

scanned_bags = []
i=0

clock = pygame.time.Clock()
running = True
last_spawn_time = 0  
spawn_interval = 2000 

bag_start_times = []  
start_time = pygame.time.get_ticks()  

while running:
    screen.fill(WHITE) 
    for belt in conveyor_belts:
        pygame.draw.line(screen, BLACK, belt[0], belt[1], 3)

    for point in points:
        pygame.draw.circle(screen, BLACK, point, 5)

    for gate in gates_entry_positions:
        pygame.draw.rect(screen, BLACK, (gate[0] - 5, gate[1] - 5, 10, 10))

    for pickup in pickup_gates_positions:
        pygame.draw.rect(screen, BLACK, (pickup[0] - 5, pickup[1] - 5, 10, 10))
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
    more_items = len(conveyor_belt)-1

    current_time = pygame.time.get_ticks()
    
    if i < len(conveyor_belt):  
        read_bag = conveyor_belt[i]  
        scanned_bags.append(read_bag)
        read_bag.set_ready()
        
        cluster_size = read_bag.cluster_size
        read_bag.set_path(paths[cluster_size])
    
        gate = gate_entries[cluster_size-2].search(read_bag)

        delay_time = 500 
        bag_start_times.append(start_time+(i*delay_time))
        
        if gate == -1:
            gate_entries[cluster_size-2].add_res(read_bag.reservation)
        else:
            gate_entries[cluster_size-2].gates[gate].set_reservation(read_bag.reservation)
            read_bag.append_path(gate_entries[cluster_size-2].gates[gate].position)
            read_bag.set_gate(gate_entries[cluster_size-2].gates[gate])
            read_bag.assigned_to_gate = True
        
        i+=1
        last_spawn_time = current_time

    still_moving = False 

    j=0
    for bag in scanned_bags:
        if bag.assigned_to_gate and bag.position == gate_entries[bag.cluster_size-2].gates[bag.gate.name-1].position:
            gate_entries[bag.cluster_size-2].gates[bag.gate.name-1].add_luggage(bag)
            pygame.draw.circle(screen, bag.color, bag.position, 8)
        elif not bag.assigned_to_gate and  abs(bag.position[0] - gate_entries[bag.cluster_size-2].position[0]) <= 10 and abs(bag.position[1] - gate_entries[bag.cluster_size-2].position[1]) <= 10:
            bag.append_path(paths_2[bag.cluster_size])
            bag.append_path(paths[bag.cluster_size])
            if bag.path: 
                bag.move()
                pygame.draw.circle(screen, bag.color, bag.position, 8)
        elif current_time >= bag_start_times[j]:
            bag.move()
            pygame.draw.circle(screen, bag.color, bag.position, 8)
            still_moving = True 
        '''

        for gate_group in gate_entries: 
            for gate in gate_group.gates: 
                if gate.is_full() and random.random() < 0.5: 
                    removed_bags = gate.clear_luggage()  
                    #scanned_bags = [bag for bag in scanned_bags if bag not in removed_bags]  
                    print(f" Gate {gate_group.name} was emptied, and {len(removed_bags)} bags disappeared!")  
        '''
        j+=1

    


    pygame.display.update()
    clock.tick(60) 

    
    if i >= len(conveyor_belt) and not still_moving:
        running = False  

pygame.quit()


2025-02-26 12:51:47.384 python[6396:499621] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-26 12:51:47.384 python[6396:499621] +[IMKInputSession subclass]: chose IMKInputSession_Modern
